# Segmenting and Clustering Neighborhoods in Toronto
We will scrape information from a Wikipedia page, read that into dataframe, do some K-means clustering and visualize with Folium!
The web page is following:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

## 1. Scraping postal code information from a web page

In [2]:
# Install and import required libraries, as in New Your clustering notebook
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [3]:
!conda install -c conda-forge beautifulsoup4 --yes 
from bs4 import BeautifulSoup

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
beautifulsoup4            4.6.3                    py35_0    conda-forge


In [4]:
# Get the page and ensure we got it 
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
page.status_code

200

In [58]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighbourhood'] 
# instantiate the dataframe
postal_codes = pd.DataFrame(columns=column_names)

soup = BeautifulSoup(page.content, 'html.parser')

# First, get the table in question
tables = soup.findChildren('table')
# We are interested in the first table only
post_table = tables[0]
rows = post_table.findChildren('tr')
# Skip the first tr (it is the header)
post_rows = rows[1:]
for row in post_rows:
    # Parse table row cells, stripping whitespace
    cells = row.findChildren('td')
    postcode = cells[0].string.strip()
    borough = cells[1].string.strip()
    for anchor in cells[1].findChildren('a'):
        borough = anchor.string.strip()
    anchorsFound = False
    for anchor in cells[2].findChildren('a'):
        neighbourhood = anchor.string.strip()
        anchorsFound = True
    if anchorsFound == False:
        neighbourhood = cells[2].string.strip() 
    if borough != 'Not assigned':
        if neighbourhood == 'Not assigned':
            neighbourhood = borough
            
        # Filter dataframes based on conditions given
        sameBorough = postal_codes[(postal_codes.PostalCode == postcode) & (postal_codes.Borough == borough)]
        if sameBorough.shape[0] > 0:
            # Already existing (postalcode, borough tuple) => replace existing list of neighbourhoods in a dataframe
            postal_codes['Neighbourhood'].replace(
                    to_replace=sameBorough['Neighbourhood'].iloc[0],
                    value=sameBorough['Neighbourhood'].iloc[0] + ', ' + neighbourhood,
                    inplace=True
            )
        else:
            # A new (postalcode, borough) tuple => append new dataframe
            postal_codes = postal_codes.append({'PostalCode': postcode,'Borough': borough,'Neighbourhood': neighbourhood}, ignore_index=True)
postal_codes.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [59]:
postal_codes.shape

(103, 3)

## 2. Getting latitude and longitude of each neighbourhood

In [7]:
!conda install -c conda-forge geocoder --yes 
import geocoder

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge


In [8]:
# The code was removed by Watson Studio for sharing.

In [60]:
# A function to return lat and long given postal code
def fetchCoordinatesByAddress(address):
    coords = None
    n_times = 0
    # Created a Bing Maps account, the other ones don't seem to work reliably...
    # loop until you get the coordinates, max 5 times
    while((coords is None) & (n_times < 5)):
        print('Trying to find {}'.format(address))
        #g = geocoder.osm('{}, {}'.format(postal_code, area))
        #coords = g.osm
        g = geocoder.bing(address, key=BING_KEY)
        coords = g.latlng
        n_times = n_times + 1
    if coords != None:
        # return coords.get('x'), coords.get('y')
        return coords[0], coords[1]
    else:
        return 0.0, 0.0

# Loop through all areas and fetch coordinates by postal code
lats=[]
longs=[]
for index, area in postal_codes.iterrows():
    lat, lon = fetchCoordinatesByAddress('{}, Toronto, Ontario'.format(area['PostalCode']))
    lats.append(lat)
    longs.append(lon)
postal_codes = postal_codes.assign(Latitude=lats, Longitude=longs)
postal_codes.head(15)


Trying to find M3A, Toronto, Ontario
Trying to find M4A, Toronto, Ontario
Trying to find M5A, Toronto, Ontario
Trying to find M6A, Toronto, Ontario
Trying to find M7A, Toronto, Ontario
Trying to find M9A, Toronto, Ontario
Trying to find M1B, Toronto, Ontario
Trying to find M3B, Toronto, Ontario
Trying to find M4B, Toronto, Ontario
Trying to find M5B, Toronto, Ontario
Trying to find M6B, Toronto, Ontario
Trying to find M9B, Toronto, Ontario
Trying to find M1C, Toronto, Ontario
Trying to find M3C, Toronto, Ontario
Trying to find M4C, Toronto, Ontario
Trying to find M5C, Toronto, Ontario
Trying to find M6C, Toronto, Ontario
Trying to find M9C, Toronto, Ontario
Trying to find M1E, Toronto, Ontario
Trying to find M4E, Toronto, Ontario
Trying to find M5E, Toronto, Ontario
Trying to find M6E, Toronto, Ontario
Trying to find M1G, Toronto, Ontario
Trying to find M4G, Toronto, Ontario
Trying to find M5G, Toronto, Ontario
Trying to find M6G, Toronto, Ontario
Trying to find M1H, Toronto, Ontario
T

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751255,-79.329895
1,M4A,North York,Victoria Village,43.729958,-79.314201
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655220,-79.361969
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722801,-79.450691
4,M7A,Queen's Park,Queen's Park,43.664486,-79.393021
5,M9A,Etobicoke,Islington Avenue,43.662743,-79.528427
6,M1B,Scarborough,"Rouge, Malvern",43.810154,-79.194603
7,M3B,North York,Don Mills North,43.749134,-79.362007
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707577,-79.310913
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657467,-79.377708


## 3. Grouping similar neighbourhoods together
We use the same kind of analysis, based on k-means clustering, that was done with New York neighborhoods in the previous lab
### 3.1 Neighbourhoods on the map

In [61]:
# We'll make a folium map of the neighborhoods. To do that, we need the address of Toronto, Ontario

torontoLat, torontoLon = fetchCoordinatesByAddress('Toronto, Ontario')
map_toronto = folium.Map(location=[torontoLat, torontoLon], zoom_start=11)

# add markers to map
for lat, lng, label in zip(postal_codes['Latitude'], postal_codes['Longitude'], postal_codes['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Trying to find Toronto, Ontario


## 3.2 Get neighbourhood venues from Foursquare

In [16]:
# The code was removed by Watson Studio for sharing.

In [62]:
VERSION = '20180605' # Foursquare API version
LIMIT=100

# This is copied from the New York lab
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            FOURSQUARE_CLIENT_ID, 
            FOURSQUARE_CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(postal_codes['Neighbourhood'], postal_codes['Latitude'], postal_codes['Longitude'])

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [63]:
# See what we got
print(toronto_venues.shape)
toronto_venues.head()

(2403, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.751255,-79.329895,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.751255,-79.329895,PetSmart,43.748639,-79.333488,Pet Store
2,Parkwoods,43.751255,-79.329895,KFC,43.754387,-79.333021,Fast Food Restaurant
3,Parkwoods,43.751255,-79.329895,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Parkwoods,43.751255,-79.329895,TTC stop #8380,43.752672,-79.326351,Bus Stop


In [64]:
# See number of venus by neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,21,21,21,21,21,21
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",2,2,2,2,2,2
Bayview Village,3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,19,19,19,19,19,19


In [66]:
# See how many neighbourhoods got venues
toronto_venues.groupby('Neighbourhood').count().shape

(101, 6)

In [68]:
postal_codes.shape

(103, 5)

In [75]:
# We have a problem - 2 neighbourhoods don't have any venues - with IDs 36 and 93. We'll just drop those!
# We will be unable to combine dataframes later if we don't do this
postal_codes.drop([36, 93], inplace=True)
postal_codes.shape

(101, 5)

## 3.3 Analyze the neighbourhoods

In [69]:
# One-hot encoding of the venues
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [70]:
toronto_onehot.shape

(2403, 265)

In [71]:
# Venue types per Neighbourhood
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Airport,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.010000,0.000000,0.00,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,0.020000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0

In [72]:
# Get the top 10 venus per neighbourhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Restaurant,Breakfast Spot,Gastropub,Deli / Bodega,Burger Joint
1,Agincourt,Coffee Shop,Supermarket,Chinese Restaurant,Pizza Place,Shanghai Restaurant,Sandwich Place,Discount Store,Sushi Restaurant,Bubble Tea Shop,Print Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Pharmacy,Park,Gym,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Pharmacy,Fried Chicken Joint,Sandwich Place,Hardware Store,Fast Food Restaurant,Liquor Store,Park
4,"Alderwood, Long Branch",Pharmacy,Gym,Sandwich Place,Dance Studio,Pub,Coffee Shop,Athletics & Sports,Bank,Pizza Place,Ethiopian Restaurant
5,"Bathurst Manor, Downsview North, Wilson Heights",IT Services,Park,Yoga Studio,Exhibit,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant
6,Bayview Village,Construction & Landscaping,Trail,Dog Run,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Exhibit
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Pizza Place,Indian Restaurant,Fast Food Restaurant,Butcher,Sandwich Place,Café,Liquor Store,Restaurant
8,Berczy Park,Steakhouse,Cosmetics Shop,Coffee Shop,Lounge,Movie Theater,Liquor Store,Belgian Restaurant,Restaurant,Beach,Pub
9,"Birch Cliff, Cliffside West",Park,General Entertainment,College Stadium,Skating Rink,Gym Pool,Gym,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


## 3.4 Cluster analysis of the neighbourhoods

In [73]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 1, 3, 0, 0, 1], dtype=int32)

In [78]:
# combine datasets
toronto_merged = postal_codes

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.751255,-79.329895,0,Park,Fast Food Restaurant,Food & Drink Shop,Pet Store,Bus Stop,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.729958,-79.314201,0,Pizza Place,Portuguese Restaurant,Intersection,Park,Farmers Market,Fast Food Restaurant,Farm,Falafel Restaurant,Exhibit,Dog Run
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655220,-79.361969,1,Coffee Shop,Breakfast Spot,Park,Thai Restaurant,Mexican Restaurant,Event Space,Café,Restaurant,Electronics Store,Food Truck
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722801,-79.450691,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Toy / Game Store,Restaurant,Accessories Store,Jewelry Store,American Restaurant,Cosmetics Shop,Fast Food Restaurant
4,M7A,Queen's Park,Queen's Park,43.664486,-79.393021,0,Coffee Shop,Park,Sandwich Place,Pub,Café,Restaurant,Chinese Restaurant,Museum,Music School,Music Store


In [79]:
toronto_merged.shape

(101, 16)

In [81]:
# Visualize the clusters - straight from New York lab

# create map
map_clusters = folium.Map(location=[torontoLat, torontoLon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [87]:
toronto_merged[toronto_merged['Cluster Labels'] == 0].shape

(74, 16)

In [82]:
# Examine cluster 0 - 74 areas (all around Toronto)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Fast Food Restaurant,Food & Drink Shop,Pet Store,Bus Stop,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
1,North York,0,Pizza Place,Portuguese Restaurant,Intersection,Park,Farmers Market,Fast Food Restaurant,Farm,Falafel Restaurant,Exhibit,Dog Run
3,North York,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Toy / Game Store,Restaurant,Accessories Store,Jewelry Store,American Restaurant,Cosmetics Shop,Fast Food Restaurant
4,Queen's Park,0,Coffee Shop,Park,Sandwich Place,Pub,Café,Restaurant,Chinese Restaurant,Museum,Music School,Music Store
7,North York,0,Coffee Shop,Burger Joint,Park,Yoga Studio,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit
8,East York,0,Fast Food Restaurant,Pizza Place,Athletics & Sports,Breakfast Spot,Café,Rock Climbing Spot,Bank,Intersection,Pharmacy,Gastropub
10,North York,0,Pizza Place,Grocery Store,Fast Food Restaurant,Japanese Restaurant,Rental Car Location,Pub,Mediterranean Restaurant,Cuban Restaurant,Cupcake Shop,Construction & Landscaping
11,Etobicoke,0,Pizza Place,Construction & Landscaping,Bank,Tea Room,Chinese Restaurant,Sandwich Place,Elementary School,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
12,Scarborough,0,Bar,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
14,East York,0,Pizza Place,Smoke Shop,Metro Station,Bank,Thrift / Vintage Store,Sushi Restaurant,Grocery Store,Arts & Crafts Store,Bar,Pet Store


In [88]:
toronto_merged[toronto_merged['Cluster Labels'] == 1].shape

(15, 16)

In [83]:
# Examine cluster 1 - 15 areas
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Breakfast Spot,Park,Thai Restaurant,Mexican Restaurant,Event Space,Café,Restaurant,Electronics Store,Food Truck
5,Etobicoke,1,Pharmacy,Bank,Skating Rink,Shopping Mall,Park,Café,Grocery Store,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Bar,Cosmetics Shop,Japanese Restaurant,Theater,Restaurant,Ramen Restaurant
29,East York,1,Trail,Clothing Store,Pool,Skating Rink,Curling Ice,Grocery Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School
40,North York,1,Coffee Shop,Airport,Food Court,Park,Yoga Studio,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
47,East Toronto,1,Café,Indian Restaurant,Sandwich Place,Italian Restaurant,Bar,Baseball Field,Movie Theater,Pub,Board Shop,Liquor Store
54,East Toronto,1,Coffee Shop,Italian Restaurant,Café,Gastropub,Park,Chinese Restaurant,Clothing Store,Latin American Restaurant,Cheese Shop,Juice Bar
66,North York,1,Park,Bank,Speakeasy,Falafel Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit,Yoga Studio
68,Central Toronto,1,Locksmith,Park,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit,Yoga Studio,Donut Shop
70,Etobicoke,1,Pizza Place,Chinese Restaurant,Sandwich Place,Coffee Shop,Middle Eastern Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space


In [89]:
toronto_merged[toronto_merged['Cluster Labels'] == 2].shape

(6, 16)

In [84]:
# Examine cluster 2 - 6 areas
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,North York,2,Asian Restaurant,Dim Sum Restaurant,Italian Restaurant,Restaurant,Fast Food Restaurant,Chinese Restaurant,Beer Store,Bike Shop,Gym,Sporting Goods Shop
23,East York,2,Sporting Goods Shop,Coffee Shop,Electronics Store,Gastropub,Record Shop,Sandwich Place,Clothing Store,Restaurant,Bank,Bike Shop
33,North York,2,Clothing Store,Park,Theater,Shopping Mall,Boutique,Liquor Store,Restaurant,Movie Theater,Baseball Field,Frozen Yogurt Shop
41,East Toronto,2,Discount Store,Park,Café,Grocery Store,Ice Cream Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
55,North York,2,Italian Restaurant,Coffee Shop,Pizza Place,Indian Restaurant,Fast Food Restaurant,Butcher,Sandwich Place,Café,Liquor Store,Restaurant
98,Etobicoke,2,Park,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Exhibit,Dog Run


In [90]:
toronto_merged[toronto_merged['Cluster Labels'] == 3].shape

(3, 16)

In [85]:
# Examine cluster 3 - 3 areas
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Fast Food Restaurant,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Exhibit
18,Scarborough,3,Gym / Fitness Center,Athletics & Sports,Construction & Landscaping,Tea Room,Park,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
56,York,3,Playground,Fast Food Restaurant,Coffee Shop,Construction & Landscaping,History Museum,Discount Store,Hobby Shop,Farmers Market,Farm,Falafel Restaurant


In [91]:
toronto_merged[toronto_merged['Cluster Labels'] == 4].shape

(3, 16)

In [86]:
# Examine cluster 4 - 3 areas
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Park,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Exhibit,Dog Run
82,Scarborough,4,Pizza Place,Shopping Mall,Bus Stop,Coffee Shop,Pharmacy,Farmers Market,Farm,Falafel Restaurant,Exhibit,Donut Shop
91,Downtown Toronto,4,Playground,Bank,Park,Candy Store,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Dumpling Restaurant
